# Introduction to the Function Calling

The most powerful feature of chat completion is the ability to call functions from the model. This allows you to create a chat bot that can interact with your existing code, making it possible to automate business processes, create code snippets, and more.

With Semantic Kernel, we simplify the process of using function calling by automatically describing your functions and their parameters to the model and then handling the back-and-forth communication between the model and your code.

Read more about it [here](https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion/function-calling).

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.23.0

### Setting Up Execution Settings

Using `FunctionChoiceBehavior.Auto()` will enable automatic function calling. There are also other options like `Required` or `None` which allow to control function calling behavior. More information about it can be found [here](https://learn.microsoft.com/en-gb/semantic-kernel/concepts/ai-services/chat-completion/function-calling/function-choice-behaviors?pivots=programming-language-csharp).

In [2]:
#pragma warning disable SKEXP0001

OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

### Providing plugins to the Kernel
Function calling needs an information about available plugins/functions. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [3]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "prompt_template_samples");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

Define your ASK. What do you want the Kernel to do?

In [4]:
var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem.";

Since we imported available plugins to Kernel and defined the ask, we can now invoke a prompt with all the provided information. 

We can run function calling with Kernel, if we are interested in result only.

In [5]:
var result = await kernel.InvokePromptAsync(ask, new(openAIPromptExecutionSettings));

Console.WriteLine(result);

Here are some delightful date ideas for Valentine's Day, presented in poetic form:

1. **Moonlit Stroll**
   On a moonlit stroll, hand in hand,  
   We tripped on the sand, oh so grand!  
   With a serenade sweet,  
   I stepped on a beet,  
   And now I’m the star of the band!  

2. **Cozy Dinner**
   In a cozy nook, just us two,  
   With candlelight flickering, oh what a view!  
   We shared a fine meal,  
   But oh, what a deal—  
   The pasta was stuck, like glue on my shoe!  
   With laughter we chewed, what a sight,  
   Our hearts felt so light, everything's right.

3. **Painting Class**
   In a class where the colors ran wild,  
   We painted like each was a child.  
   With splatters and drips,  
   And some funny quips,  
   Our canvases looked quite beguiled!  

4. **Picnic in the Park**
   In the park where the sun shines so bright,  
   We spread out our blanket, what a sight!  
   With sandwiches stacked,  
   And a squirrel who snacked,  
   On our chips, he declared, “

But we can also run it with `IChatCompletionService` to have an access to `ChatHistory` object, which allows us to see which functions were called as part of a function calling process. Note that passing a Kernel as a parameter to `GetChatMessageContentAsync` method is required, since Kernel holds an information about available plugins.

In [6]:
using Microsoft.SemanticKernel.ChatCompletion;

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var chatHistory = new ChatHistory();

chatHistory.AddUserMessage(ask);

var chatCompletionResult = await chatCompletionService.GetChatMessageContentAsync(chatHistory, openAIPromptExecutionSettings, kernel);

Console.WriteLine($"Result: {chatCompletionResult}\n");
Console.WriteLine($"Chat history: {JsonSerializer.Serialize(chatHistory)}\n");

Result: Here are some charming date ideas for Valentine’s Day, each expressed in poetic form:

1. **Candle-lit Dinner**:
   In a garden where fireflies dance,  
   Two lovers sat lost in romance.  
   With candles aglow,  
   And a soft, gentle flow,  
   Till the soup spilled—oh, what a mischance!  

2. **Sunset Hike**:
   On a hill where the sunsets glow,  
   We hiked hand in hand, nice and slow.  
   But my shoe had a tear,  
   And I tripped on a hare,  
   Now I’m rolling down, yelling “Whoa!”  

3. **Art Session**:
   In a studio filled with cheer,  
   We splashed paint without any fear.  
   With a brush in my hand,  
   I made a big band,  
   But it looked more like a cow in a sphere!  

4. **Cozy Movie Night**:
   In a blanket fort snug and tight,  
   We settled in for a movie night.  
   With popcorn galore,  
   And a snack attack score,  
   But the cat stole our bowl—what a fright!  

Feel free to choose any of these ideas to make your Valentine's Day special!

Chat hi